In [23]:
%reset -f
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
# from pyimagesearch.lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

# initialize the number of epochs to train for, initial learning rate,
# and batch size
epochs = 100
batch_size = 32
monitor='val_loss'
es_patience=7
rlr_patience=3
img_width, img_height = 60,60

In [24]:
def get_dataset(dataset='./insectrec/created_data/impy_crops_export/'):
    # initialize the data and labels
    print(" loading images...")
    data = []
    labels = []

    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(dataset)))
    random.seed(42)
    random.shuffle(imagePaths)

    # loop over the input images
    for imagePath in imagePaths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (img_width, img_height))
        image = img_to_array(image)
        data.append(image)

        # extract the class label from the image path and update the
        # labels list
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0

    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    (trainX, testX, trainY, testY) = train_test_split(data,
        labels, test_size=0.2, random_state=42)
    return trainX, testX, trainY, testY, labels

_, testX, _, testY, _ = get_dataset(dataset='./insectrec/created_data/impy_crops_export/')
trainX, _, trainY, _, labels = get_dataset(dataset='./insectrec/created_data/images_augmented/')

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=6)
testY = to_categorical(testY, num_classes=6)

 loading images...
 loading images...


In [25]:
# testY.shape

In [26]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, zoom_range=0.3,
    horizontal_flip=True, vertical_flip=True,fill_mode="nearest")

In [27]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau, CSVLogger

log_dir = './insectrec/created_data/logs/'
top_weights_path = f'./insectrec/created_data/weights/model_DenseNet_{img_width}.h5'

callbacks_list = [ModelCheckpoint(monitor =  monitor,
                                  filepath =  top_weights_path,
                                  save_best_only = False,
                                  save_weights_only = False,
                                  verbose = 1),
                  EarlyStopping(monitor =  monitor,
                                patience =  es_patience,
                                verbose = 1),
                  ReduceLROnPlateau(monitor =  monitor,
                                    factor = 0.1,
                                    patience =  rlr_patience,
                                    verbose = 1),
                # CSVLogger(filename =  logfile),
                  TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)]


In [31]:
base_model = DenseNet121(include_top=True, weights=None, 
                         input_shape=(img_width,img_height,3))
# add a global spatial average pooling layer
x = base_model.output
# x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
FH = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size,
    epochs=epochs, verbose=1, callbacks=callbacks_list)

Epoch 1/100
1909/1910 [============================>.] - ETA: 1s - loss: 1.4243 - accuracy: 0.4266
Epoch 00001: saving model to ./insectrec/created_data/weights/model_DenseNet_60.h5
1910/1910 [==============================] - 1950s 1s/step - loss: 1.4242 - accuracy: 0.4266 - val_loss: 1.3683 - val_accuracy: 0.4522
Epoch 2/100
1909/1910 [============================>.] - ETA: 1s - loss: 1.2277 - accuracy: 0.5243
Epoch 00002: saving model to ./insectrec/created_data/weights/model_DenseNet_60.h5
1910/1910 [==============================] - 2015s 1s/step - loss: 1.2275 - accuracy: 0.5243 - val_loss: 1.4611 - val_accuracy: 0.4434
Epoch 3/100
1909/1910 [============================>.] - ETA: 0s - loss: 1.1269 - accuracy: 0.5703
Epoch 00003: saving model to ./insectrec/created_data/weights/model_DenseNet_60.h5
1910/1910 [==============================] - 1932s 1s/step - loss: 1.1270 - accuracy: 0.5703 - val_loss: 1.0850 - val_accuracy: 0.6001
Epoch 4/100
1909/1910 [==========================